In [1]:
import torch
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, AutoConfig
import os
from datasets import load_from_disk
os.environ['TRANSFORMERS_CACHE'] = '/scratch/workspace/wenlongzhao_umass_edu-analyze/transformers_cache'



In [7]:
data_feedback = load_from_disk("/scratch/workspace/wenlongzhao_umass_edu-analyze/Bidirectional-Teacher-Student-Communication-for-LLM-Alignment/datasets/gsm8k/feedback")
data_feedback


Dataset({
    features: ['question', 'answer'],
    num_rows: 5082
})

In [8]:
data_train = load_from_disk("/scratch/workspace/wenlongzhao_umass_edu-analyze/Bidirectional-Teacher-Student-Communication-for-LLM-Alignment/datasets/gsm8k/train")
data_train

Dataset({
    features: ['question', 'answer'],
    num_rows: 896
})

In [9]:
import pandas as pd

In [36]:
path = '/scratch/workspace/wenlongzhao_umass_edu-analyze/Bidirectional-Teacher-Student-Communication-for-LLM-Alignment/outputs/gsm8k/LLaMA8B/generated_outputs.json'
df = pd.read_json(path)

In [11]:
df.head()

,input,output
0,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistantuser\n\nB...
1,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistantuser\n\nB...
2,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistantuser\n\nB...
3,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistantuser\n\nB...
4,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistantuser\n\nB...


In [33]:
def get_answer_dataset(row):
    return row['answer'].split('####')[-1]
def get_answer_model(row):
    return row['output'].split('####')[-1]

In [37]:
df.drop(columns=['score'],inplace=True)
df['model_answer']=df.apply(get_answer_model,axis=1)

In [38]:
df.head()

,input,output,model_answer
0,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistantuser\n\nB...,108
1,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistantuser\n\nB...,3
2,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistantuser\n\nB...,50%
3,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistantuser\n\nB...,3
4,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistantuser\n\nB...,159


In [44]:
from evaluate import load
from tqdm import tqdm
exact_match_metric = load("exact_match")

In [45]:
# results = exact_match_metric.compute(predictions=[df.iloc[3]['model_answer']], references=[data_feedback["answer"][3].split("####")[-1]])

In [45]:
feedback = pd.DataFrame(data_feedback)
feedback.head()

,question,answer
0,Jordan read 120 French novels last holiday. Hi...,Alexandre read 120 * 1/10 = <<120*1/10=12>>12 ...
1,Samantha’s last name has three fewer letters t...,"There are 4 letters in Jamie’s last name, so B..."
2,Anna is making gingerbread cookies. She gives ...,The first step to find the total number of gin...
3,Frank has three less than half the number of c...,Mike has 4 * 3 = <<4*3=12>>12 cookies.\nFrank ...
4,"Mr. Smith takes his wife, his parents, and his...",The buffet cost for Mr. Smith and his wife is ...


In [46]:
feedback['GT_Answer']=feedback.apply(get_answer_dataset,axis=1)

In [47]:
feedback.head()

,question,answer,GT_Answer
0,Jordan read 120 French novels last holiday. Hi...,Alexandre read 120 * 1/10 = <<120*1/10=12>>12 ...,108
1,Samantha’s last name has three fewer letters t...,"There are 4 letters in Jamie’s last name, so B...",7
2,Anna is making gingerbread cookies. She gives ...,The first step to find the total number of gin...,50
3,Frank has three less than half the number of c...,Mike has 4 * 3 = <<4*3=12>>12 cookies.\nFrank ...,3
4,"Mr. Smith takes his wife, his parents, and his...",The buffet cost for Mr. Smith and his wife is ...,159


In [57]:
# for i in tqdm(range(3)):
#     df['score']=exact_match_metric.compute(predictions=[df.iloc[i]['model_answer']], references=[feedback.iloc[i]["GT_Answer"]])['exact_match']

In [58]:
df.head()

,input,output,model_answer,score
0,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistantuser\n\nB...,108,0.0
1,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistantuser\n\nB...,3,0.0
2,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistantuser\n\nB...,50%,0.0
3,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistantuser\n\nB...,3,0.0
4,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistantuser\n\nB...,159,0.0


In [59]:
# score= exact_match_metric.compute(predictions=df['model_answer'].tolist(), references=feedback["GT_Answer"].tolist())

In [60]:
df['score'] = (df['model_answer'] == feedback['GT_Answer']).astype(int)

In [61]:
# from tqdm import tqdm
# for i in tqdm(range(10)):
    

In [62]:
df.head(10)

,input,output,model_answer,score
0,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistantuser\n\nB...,108,1
1,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistantuser\n\nB...,3,0
2,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistantuser\n\nB...,50%,0
3,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistantuser\n\nB...,3,1
4,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistantuser\n\nB...,159,1
5,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistantuser\n\nB...,9474,1
6,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistantuser\n\nB...,75,1
7,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistantuser\n\nB...,20,1
8,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistantuser\n\nB...,600,1
9,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistantuser\n\nB...,4,1


In [63]:
df.drop(columns=['model_answer'],inplace=True)

In [64]:
df.head()

,input,output,score
0,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistantuser\n\nB...,1
1,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistantuser\n\nB...,0
2,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistantuser\n\nB...,0
3,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistantuser\n\nB...,1
4,<|begin_of_text|><|start_header_id|>system<|en...,system\n\nYou are a helpful assistantuser\n\nB...,1


In [65]:
# df.to_json("/scratch/workspace/wenlongzhao_umass_edu-analyze//Bidirectional-Teacher-Student-Communication-for-LLM-Alignment/outputs/gsm8k/LLaMA8B/generated_outputs1.json", orient='records', lines=True)

In [66]:
records = df.to_dict(orient='records')

In [67]:
import json
formatted_json = json.dumps(records, indent=4)


In [68]:
with open("/scratch/workspace/wenlongzhao_umass_edu-analyze/Bidirectional-Teacher-Student-Communication-for-LLM-Alignment/outputs/gsm8k/LLaMA8B/generated_outputs1.json", "w") as f:
    f.write(formatted_json)

In [69]:
df['score'].sum()

np.int64(4302)

In [72]:
4302/5082


0.846517119244392

In [71]:
df.shape[0]

5082